In [42]:
import pandas as pd
import numpy as np

# 데이터 불러오기
data = pd.read_csv('WallThining_data.csv')
data = data.drop('Moment', axis=1)
# 데이터 확인
print(data.head())

  Thining Part Moment Direction Thining Rate Boundary Type  Inner Pressure  \
0        Extra              IPO          20%    Limit Load          22.070   
1        Extra              IPO          20%    Limit Load          19.863   
2        Extra              IPO          20%    Limit Load          17.656   
3        Extra              IPO          20%    Limit Load          15.449   
4        Extra              IPO          20%    Limit Load          13.242   

   Multiplier  
0    0.754368  
1    0.889164  
2    1.000607  
3    1.022539  
4    0.970476  


In [43]:
from sklearn.preprocessing import MinMaxScaler

# Min-Max Scaler 객체 생성
scaler = MinMaxScaler()

# 'Inner Pressure' 열 선택 및 Reshape (-1, 1) 필요 (scaler는 2D 배열을 요구함)
inner_Pressure = data['Inner Pressure'].values.reshape(-1, 1)

# Min-Max Normalization 적용
data['Inner Pressure'] = scaler.fit_transform(inner_Pressure)

# 결과 확인
print(data.head())


  Thining Part Moment Direction Thining Rate Boundary Type  Inner Pressure  \
0        Extra              IPO          20%    Limit Load             1.0   
1        Extra              IPO          20%    Limit Load             0.9   
2        Extra              IPO          20%    Limit Load             0.8   
3        Extra              IPO          20%    Limit Load             0.7   
4        Extra              IPO          20%    Limit Load             0.6   

   Multiplier  
0    0.754368  
1    0.889164  
2    1.000607  
3    1.022539  
4    0.970476  


In [44]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Categorical 변수 인코딩
# le = LabelEncoder()
# data['Thining Part'] = le.fit_transform(data['Thining Part'])
# data['Moment Direction'] = le.fit_transform(data['Moment Direction'])
# data['Boundary Type'] = le.fit_transform(data['Boundary Type'])

data = pd.get_dummies(data, columns=['Thining Part','Moment Direction','Boundary Type'])

# Numerical 변수 전처리 (Inner pressure)
data['Thining Rate'] = data['Thining Rate'].str.rstrip('%').astype('int')

# 데이터 분할
X = data.drop('Multiplier', axis=1)
y = data['Multiplier']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [47]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, GRU, SimpleRNN, Conv1D, MaxPooling1D, Flatten
from tensorflow.keras.utils import to_categorical


models = {
    'MLP': Sequential([
        Dense(64, activation='relu', input_shape=(10,)),
        Dense(32, activation='relu'),
        Dense(1, activation='sigmoid')
    ]),
    'CNN': Sequential([
        # Conv1D는 3D 입력이 필요하므로 (샘플 수, 타임스텝, 특성) 형태로 변환 필요
        Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(10, 1)),
        MaxPooling1D(pool_size=2),
        Flatten(),
        Dense(50, activation='relu'),
        Dense(1, activation='sigmoid')
    ]),
    'RNN': Sequential([
        SimpleRNN(50, activation='relu', input_shape=(10, 1)),
        Dense(1, activation='sigmoid')
    ]),
    'LSTM': Sequential([
        LSTM(50, activation='relu', input_shape=(10, 1)),
        Dense(1, activation='sigmoid')
    ]),
    'GRU': Sequential([
        GRU(50, activation='relu', input_shape=(10, 1)),
        Dense(1, activation='sigmoid')
    ])
}

# MAPE와 SMAPE 함수 정의
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def symmetric_mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs(y_true - y_pred) / ((np.abs(y_true) + np.abs(y_pred)) / 2)) * 100




# 모델 컴파일 및 학습
for name, model in models.items():
    model.compile(optimizer='adam', loss='mean_squared_error')
    if name == 'MLP':
        model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=0)
        y_pred = model.predict(X_test)
    else:
        X_train_transformed = X_train.to_numpy().reshape((X_train.shape[0], X_train.shape[1], 1))
        model.fit(X_train_transformed, y_train, epochs=10, batch_size=32, verbose=0)
        X_test_transformed = X_test.to_numpy().reshape((X_test.shape[0], X_test.shape[1], 1))
        y_pred = model.predict(X_test_transformed)

    # 실제 값과 예측 값의 차이를 계산하여 평가 지표를 구함
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    mape = mean_absolute_percentage_error(y_test, y_pred)
    smape = symmetric_mean_absolute_percentage_error(y_test, y_pred)   
    r2 = r2_score(y_test, y_pred)

    print(f'{name} - MAE: {mae:.4f}, MSE: {mse:.4f}, RMSE: {rmse:.4f}, MAPE: {mape:.4f}, SMAPE: {smape:.4f}, R2: {r2:.4f}')

MLP - MAE: 0.0964, MSE: 0.0129, RMSE: 0.1134, MAPE: 21.1525, SMAPE: 19.0993, R2: 0.3329
CNN - MAE: 0.0779, MSE: 0.0089, RMSE: 0.0944, MAPE: 21.9159, SMAPE: 20.4267, R2: 0.5374
RNN - MAE: 0.1038, MSE: 0.0150, RMSE: 0.1226, MAPE: 20.9106, SMAPE: 18.7677, R2: 0.2193
